In [2]:
# import everything we may need
import mysql.connector
from mysql.connector import errorcode
import config
import requests
import time
import json
import pandas as pd
%load_ext autoreload
%autoreload 2
import json

In [66]:
#get data from box office mojo as json file
f = open('mojo_data.json')
data = json.load(f)

In [97]:
# connect to AWS
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password
)

# Establish cursor
cursor = cnx.cursor()

In [4]:
## function to close the connection
def close_connections():
    cursor.close()
    conn.close()

In [5]:
#name the db
db_name = 'Mod_1_Project'

In [6]:
# create database in AWS
def create_database(cursor, database_name):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database_name))
    except mysql.connector.Error as err:
        ## Catch the error if an error occurs.
        print("Failed creating database: {}".format(err))
        exit(1)
        
try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exist.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

In [67]:
#get list of movies from box office mojo data
titles = [x['title'] for x in data]
titles

['The Boss Baby',
 "A Dog's Purpose",
 'Jigsaw',
 'Guardians of the Galaxy Vol. 2',
 'Split',
 'Jumanji:Welcome to the Jungle',
 'All Eyez on Me',
 'Downsizing',
 'Annabelle:Creation',
 'Despicable Me 3',
 'Spider-Man:Homecoming',
 "Molly's Game",
 'The LEGO Batman Movie',
 'American Assassin',
 'Geostorm',
 'Star Wars:The Last Jedi',
 'Transformers:The Last Knight',
 'Baby Driver',
 'Ghost in the Shell (2017)',
 'Atomic Blonde',
 'Beauty and the Beast (2017)',
 'Justice League',
 'Logan Lucky',
 "Tyler Perry's Boo 2! A Madea Halloween",
 'Rings',
 'Victoria and Abdul',
 'Smurfs:The Lost Village',
 'I, Tonya',
 'Baywatch',
 'The Shape of Water',
 'Rough Night',
 'Thor:Ragnarok',
 'Valerian and the City of a Thousand Planets',
 'Alien:Covenant',
 'American Made',
 'Captain Underpants:The First Epic Movie',
 'The Dark Tower',
 'Get Out',
 'Lady Bird',
 'Coco',
 'The Post',
 'John Wick:Chapter Two',
 'The Foreigner',
 'The Fate of the Furious',
 'Leap!',
 'Kidnap (2017)',
 'Pirates of the

In [69]:
key = config.apiKey
url = 'http://www.omdbapi.com/?apikey='

# Make call to API
# Iterate over list of titles (from box office mojo) to retrieve all

def OMDB_call(titles):
    all_titles = []
    for title in titles: 
        parameters = {'t': {title}}
        response = requests.get(url+key, params=parameters)
        data = response.json()
        all_titles.append(data)
        time.sleep(.5)
    return all_titles

movie_dict = OMDB_call(titles[:100])

In [71]:
movie_dict[0]

{'Title': 'The Boss Baby',
 'Year': '2017',
 'Rated': 'PG',
 'Released': '31 Mar 2017',
 'Runtime': '97 min',
 'Genre': 'Animation, Adventure, Comedy, Family, Fantasy',
 'Director': 'Tom McGrath',
 'Writer': 'Michael McCullers, Marla Frazee (based on the book by)',
 'Actors': 'Alec Baldwin, Steve Buscemi, Jimmy Kimmel, Lisa Kudrow',
 'Plot': 'A suit-wearing, briefcase-carrying baby pairs up with his 7-year old brother to stop the dastardly plot of the CEO of Puppy Co.',
 'Language': 'English, Spanish',
 'Country': 'USA',
 'Awards': 'Nominated for 1 Oscar. Another 1 win & 19 nominations.',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BMTg5MzUxNzgxNV5BMl5BanBnXkFtZTgwMTM2NzQ3MjI@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.3/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '52%'},
  {'Source': 'Metacritic', 'Value': '50/100'}],
 'Metascore': '50',
 'imdbRating': '6.3',
 'imdbVotes': '94,906',
 'imdbID': 'tt3874544',
 'Type': 'movie',
 'DVD': '25 Jul

In [ ]:
#function to create tables in our database
def create_table(dict_of_tables, db_name):
    for table_name in dict_of_tables:
        table_query = dict_of_tables[table_name]
        cursor.execute("USE {}".format(db_name))
        try:
            print("Creating table {}: ".format(table_name), end='')
            cursor.execute(table_query)
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
                print("already exists.")
            else:
                print(err.msg)
        else:
            print("OK")


In [46]:
#Create tables to input data
TABLES = {}
TABLES['main_movie_table'] = (
    "CREATE TABLE main_movie_table ("
    "  movie_id int NOT NULL AUTO_INCREMENT,"
    "  title text NOT NULL,"
    "  budget int(10),"
    "  release_date date,"
    "  award_wins int(3),"
    "  award_nomination int(3),"
    "  RT_rating int(2),"
    "  IMDB_rating real,"
    "  PRIMARY KEY (movie_id)"
    ") ENGINE=InnoDB")

TABLES['genres'] = (
    "CREATE TABLE genres ("
    "  genre_id int NOT NULL,"
    "  genre text,"
    "  PRIMARY KEY (genre_id)"
    ") ENGINE=InnoDB")

TABLES['key_words'] = (
    "CREATE TABLE key_words ("
    "  keyword_id int NOT NULL,"
    "  key_words text,"
    "  PRIMARY KEY (keyword_id)"
    ") ENGINE=InnoDB")

TABLES['movies_genres'] = (
    "CREATE TABLE movies_genres ("
      " movie_id int NOT NULL,"
      " genre_id int NOT NULL"
      ") ENGINE=InnoDB")

TABLES['movies_keywords'] = (
    "CREATE TABLE movies_keywords ("
      " movie_id int NOT NULL,"
      " keyword_id text NOT NULL"
      ") ENGINE=InnoDB")

TABLES['actors'] = (
    "CREATE TABLE actors ("
    " actor_id int NOT NULL AUTO_INCREMENT," 
    " actor_name text NOT NULL,"
    " PRIMARY KEY (actor_id)"
    ") ENGINE=InnoDB")

TABLES['movie_actors'] = (
    "CREATE TABLE movie_actors("
    " movie_id int NOT NULL,"
    " actor_id int NOT NULL"
    ") ENGINE=InnoDB")

create_table(TABLES, 'Mod_1_Project')

Creating table main_movie_table: OK
Creating table genres: already exists.
Creating table key_words: already exists.
Creating table movies_genres: already exists.
Creating table movies_keywords: already exists.
Creating table actors: already exists.
Creating table movie_actors: already exists.


In [ ]:
#insert data into tables
def add_movies_toDB():
    for movie in movie_dict:
        ('''insert into main_movie_table (title, release_date, award_wins, award_nomination, 
        RT_rating, IMDB_rating) values(%s,%s,%s,%s,%s,%s)''',
        movie['Title'], movie['Released'], movie['Awards'], 
        movie['Rating'][1]['Value'].replace('%', ''), movie['imdbRating']) 
        conn.commit
         
def add_genres_toDB():
    for movie in movie_dict:
        ('''insert into genres (genre) values(%s)''', movie['Genre'])
        conn.commit
            
## Drop all tables to refresh
def drop_everything():
    cursor.execute("""DROP TABLE Reviews;""")
    cnx.commit()
    cursor.execute("""DROP TABLE Businesses;""")
    cnx.commit()

In [21]:
import sys
def get_movie_id(title):
    title = title.replace(r"'",r"\'")
    cursor.execute(f"""SELECT movie_id FROM main_movie_table WHERE title = '{title}';""")
    return cursor.fetchone()

def add_actors_toDB():
    idx = 0
    sys.stdout.write("INSERTING ACTORS")
    sys.stdout.flush()
    for movie in movie_dict:
        try:
            for actor in movie['Actors'].split(', '):
                actor = actor.replace(r"'",r"\'")
                actor_insert_q = f"""INSERT INTO actors(actor_name) VALUES('{actor}');"""
                #get id of movie
                movie_id = get_movie_id(movie['Title'])
                #get actor_id if already present in database
                cursor.execute(f"""SELECT actor_id FROM actors WHERE actor_name = '{actor}';""")
                actor_id = cursor.fetchone()
                #if actor not present, insert and get id back
                if actor_id is None:
                    cursor.execute(actor_insert_q)
                    cursor.execute("""SELECT LAST_INSERT_ID();""")
                    actor_id = cursor.fetchone()
                    cnx.commit()
                #add actor + movie to movie_actors
                cursor.execute(f"""INSERT INTO movie_actors(movie_id, actor_id) VALUES ('{movie_id}', '{actor_id}');""")
                cnx.commit()
                idx += 1
                if idx % 112 == 0:
                    sys.stdout.write(".")
                    sys.stdout.flush()
        except KeyError:
            print(movie)
    sys.stdout.write("OK")
    sys.stdout.flush()
add_actors_toDB()  

INSERTING ACTORS{'Response': 'False', 'Error': 'Movie not found!'}
{'Response': 'False', 'Error': 'Movie not found!'}
{'Response': 'False', 'Error': 'Movie not found!'}
{'Response': 'False', 'Error': 'Movie not found!'}
.{'Response': 'False', 'Error': 'Movie not found!'}
{'Response': 'False', 'Error': 'Movie not found!'}
{'Response': 'False', 'Error': 'Movie not found!'}
{'Response': 'False', 'Error': 'Movie not found!'}
{'Response': 'False', 'Error': 'Movie not found!'}
{'Response': 'False', 'Error': 'Movie not found!'}
{'Response': 'False', 'Error': 'Movie not found!'}
.{'Response': 'False', 'Error': 'Movie not found!'}
.OK

In [11]:
#insert keywords into database
movie_keywords = pd.read_csv('improved_import_movies.csv')
keywords = movie_keywords['keyword'].unique()
cursor.execute("""USE Mod_1_Project;""")
for keyword in keywords:
    insert_keyword_q = f"""INSERT INTO key_words(key_words) VALUES ('{keyword}');"""
    cursor.execute(insert_keyword_q)
    cnx.commit()

In [118]:
from urllib.parse import urlencode
from imdb import IMDb
import sys

idx = 0
sys.stdout.write('[')
sys.stdout.flush()
ia = IMDb()
s = requests.Session()
movie_keywords = pd.read_csv('improved_import_movies.csv')
movie_keywords = movie_keywords[movie_keywords['release_date'] < '2017-01-01']
titles = movie_keywords['title']
expanded_movie_data = []
url_stem = 'http://www.omdbapi.com/?apikey=' + apiKey + '&'

for title in titles:
    url_title = urlencode({'t':title})
    omdb_data = json.loads(s.get(url_stem+url_title).text)
    time.sleep(1)
    try:
        win = list(get_awards(title,ia).keys())[0]
        nom = list(get_awards(title,ia).keys())[1]
        award_wins = get_awards(title,ia)[win]
        award_noms = get_awards(title,ia)[nom]
    except:
        award_wins = 'NULL'
        award_noms = 'NULL'
    expanded_movie_data.append({
        'title': title.replace(r"'", r"\'"),
        'award_wins': award_wins,
        'award_nomination': award_noms,
        'RT_rating': [rating['Value'][:-1] for rating in omdb_data['Ratings'] if rating['Source'] == 'Rotten Tomatoes'],
        'IMDB_rating': [rating['Value'][:3] for rating in omdb_data['Ratings'] if rating['Source'] == 'Internet Movie Database']
    })
    idx += 1
    if idx % 17 == 0:
        sys.stdout.write('=')
        sys.stdout.flush()
sys.stdout.write(']')
sys.stdout.flush()
expanded_movie_data
# "  title text NOT NULL,"
#     "  budget int(10),"
#     "  release_date date,"
#     "  award_wins int(3),"
#     "  award_nomination int(3),"
#     "  RT_rating int(2),"
#     "  IMDB_rating real,"

[=======================================

2019-07-31 15:27:31,396 CRITICAL [imdbpy] /Users/brendan.gilroy/anaconda3/envs/learn-env/lib/python3.7/site-packages/imdb/_exceptions.py:34: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/find?q=Reservoir+Dogs&s=tt', 'proxy': '', 'exception type': 'IOError', 'original exception': timeout('The read operation timed out')},); kwds: {}
Traceback (most recent call last):
  File "/Users/brendan.gilroy/anaconda3/envs/learn-env/lib/python3.7/site-packages/imdb/parser/http/__init__.py", line 220, in retrieve_unicode
    content = response.read()
  File "/Users/brendan.gilroy/anaconda3/envs/learn-env/lib/python3.7/http/client.py", line 454, in read
    return self._readall_chunked()
  File "/Users/brendan.gilroy/anaconda3/envs/learn-env/lib/python3.7/http/client.py", line 564, in _readall_chunked
    value.append(self._safe_read(chunk_left))
  File "/Users/brendan.gilroy/anaconda3/envs/learn-env/lib/python3.7/http/client.py", line 61

===============]

[{'title': 'Forrest Gump',
  'award_wins': 47,
  'award_nomination': 68,
  'RT_rating': ['72'],
  'IMDB_rating': ['8.8']},
 {'title': 'A League of Their Own',
  'award_wins': 6,
  'award_nomination': 13,
  'RT_rating': ['78'],
  'IMDB_rating': ['7.2']},
 {'title': 'Fargo',
  'award_wins': 59,
  'award_nomination': 203,
  'RT_rating': ['93'],
  'IMDB_rating': ['8.1']},
 {'title': 'There Will Be Blood',
  'award_wins': 109,
  'award_nomination': 136,
  'RT_rating': ['91'],
  'IMDB_rating': ['8.2']},
 {'title': 'Blue Velvet',
  'award_wins': 18,
  'award_nomination': 18,
  'RT_rating': ['94'],
  'IMDB_rating': ['7.8']},
 {'title': 'Walking Tall',
  'award_wins': 'NULL',
  'award_nomination': 'NULL',
  'RT_rating': ['26'],
  'IMDB_rating': ['6.3']},
 {'title': 'W.',
  'award_wins': 1,
  'award_nomination': 7,
  'RT_rating': ['58'],
  'IMDB_rating': ['6.3']},
 {'title': 'Serial Mom',
  'award_wins': 0,
  'award_nomination': 1,
  'RT_rating': ['63'],
  'IMDB_rating': ['6.7']},
 {'title': 'Di

In [18]:
apiKey = config.apiKey
for title in titles:
    url_title = urlencode({'t':title})
    url_stem = 'http://www.omdbapi.com/?apikey='+apiKey+'&'
    omdb_data = json.loads(requests.get(url_stem+url_title).text)
    print(omdb_data['Awards'])

Won 3 Oscars. Another 51 wins & 197 nominations.
Won 1 Oscar. Another 122 wins & 181 nominations.


In [111]:
from bs4 import BeautifulSoup
from imdb import IMDb
import inspect

def get_awards(movie_title, ia):
    try:
        movie = ia.search_movie(movie_title)[0]
    except IndexError:
        return None
    url_stem = 'https://www.imdb.com/title/tt'
    soup = BeautifulSoup(requests.get(url_stem+movie.getID()+'/awards').text)
    try:
        awards_text = soup.select('#main .desc')[0].string
    except IndexError:
        return None
    awards_words = awards_text.split()
    try:
        return {awards_words[3]:int(awards_words[2]), awards_words[6]:int(awards_words[5])}
    except IndexError:
        return None
ia = IMDb()
for title in titles:
    print(get_awards(title, ia))

{'wins': 47, 'nominations': 68}
{'wins': 6, 'nominations': 13}
{'wins': 59, 'nominations': 203}


KeyboardInterrupt: 

In [73]:
titles

['The Boss Baby',
 "A Dog's Purpose",
 'Jigsaw',
 'Guardians of the Galaxy Vol. 2',
 'Split',
 'Jumanji:Welcome to the Jungle',
 'All Eyez on Me',
 'Downsizing',
 'Annabelle:Creation',
 'Despicable Me 3',
 'Spider-Man:Homecoming',
 "Molly's Game",
 'The LEGO Batman Movie',
 'American Assassin',
 'Geostorm',
 'Star Wars:The Last Jedi',
 'Transformers:The Last Knight',
 'Baby Driver',
 'Ghost in the Shell (2017)',
 'Atomic Blonde',
 'Beauty and the Beast (2017)',
 'Justice League',
 'Logan Lucky',
 "Tyler Perry's Boo 2! A Madea Halloween",
 'Rings',
 'Victoria and Abdul',
 'Smurfs:The Lost Village',
 'I, Tonya',
 'Baywatch',
 'The Shape of Water',
 'Rough Night',
 'Thor:Ragnarok',
 'Valerian and the City of a Thousand Planets',
 'Alien:Covenant',
 'American Made',
 'Captain Underpants:The First Epic Movie',
 'The Dark Tower',
 'Get Out',
 'Lady Bird',
 'Coco',
 'The Post',
 'John Wick:Chapter Two',
 'The Foreigner',
 'The Fate of the Furious',
 'Leap!',
 'Kidnap (2017)',
 'Pirates of the

In [82]:
import re

def find_year_in_title(title):
    year = re.findall('(201\d)', title)
    print(type(year))
    if year:
        return year[0], title.replace("("+year[0]+")", "")
find_year_in_title(titles[17])

<class 'list'>


TypeError: '<' not supported between instances of 'str' and 'int'

In [136]:
import math

df = pd.DataFrame(expanded_movie_data)
df['IMDB_rating'] = df['IMDB_rating'].apply(lambda x: x[0] if len(x) > 0 else math.nan )
df['RT_rating'] = df['RT_rating'].apply(lambda x: x[0] if len(x) > 0 else math.nan)
df

,IMDB_rating,RT_rating,award_nomination,award_wins,title
0,8.8,72,68,47,Forrest Gump
1,7.2,78,13,6,A League of Their Own
2,8.1,93,203,59,Fargo
3,8.2,91,136,109,There Will Be Blood
4,7.8,94,18,18,Blue Velvet
5,6.3,26,NULL,NULL,Walking Tall
6,6.3,58,7,1,W.
7,6.7,63,1,0,Serial Mom
8,6.0,44,7,1,Divine Secrets of the Ya-Ya Sisterhood
9,7.1,73,4,7,My Dog Skip
